This is a program for retrieving all pull requests from the GitHub organization Ramda.

In [22]:
import requests
import json
import os

def getPullRequestsForOrg(organization):
    
    # Pull request parameters
    auth=('user', os.environ.get('GITHUBTOKEN'))
    state = 'all'
    perPage= '100'
    
    # For the given organization, get repositories
    repoUrl = "https://api.github.com/orgs/" +organization+ "/repos"
    response = requests.get(repoUrl, auth=auth)
    
    # Check that the response was successful
    if response.status_code != requests.codes.ok: 
        print("Repository query unsuccessful: " + str(response.status_code))
        return None
    
    jsonRepos=response.json() # jsonRepos is a list of dictionaries, where each dictionary represents a repository

    pullList = [] # the list of all pull requests, each of which will be a dictionary

    # Loop over all repositories for this organization and for each one, get all pull requests
    for repo in jsonRepos:
        pullUrl = "https://api.github.com/repos/" + repo["full_name"] + "/pulls"
        payLoad = {'state':state,'per_page':perPage} # set parameters for retrieving the pull requests
        getPullRequests(pullUrl, payLoad, pullList, auth)
        
    return pullList 
    
def getPullRequests(pullUrl, params, pullList, auth):

    # Initialize the list used for pagination
    linkList = []
    
    # Get the pull requests for this URL
    pullResponse = requests.get(pullUrl, params=params, auth=auth)
    
    # Check that the call to the GitHub API was successful
    if pullResponse.status_code != requests.codes.ok:
        print("Pull request query unsuccessful: " + str(pullResponse.status_code))
        return None

    # Turn the response into JSON for parsing
    jsonPulls = pullResponse.json()
    # Add this set of pulls to the list of found pulls so far
    pullList.extend(jsonPulls)
        
    # Now, if there was more than one page of results, call the API again to get the rest
    link = pullResponse.headers.get('link') # the HTTP headers will contain a link (string) if data was returned
    if link:
        linkList = link.split(',')
        # Iterate over the list of links to find the next page, if one is available
        for url in linkList:
            # If this particular link contains the "next" relationship, parse out the URL and call this function again
            if 'rel="next"' in url:
                nextUrl = url.replace('; rel="next"','')
                nextUrl = nextUrl.strip(' <>')
                getPullRequests(nextUrl, None, pullList, auth) # No params this time since they are part of the link already

                

resultList = getPullRequestsForOrg("ramda")

if resultList:
    print("Total number of pull requests in the Ramda organization: " + str(len(resultList)))
else:
    print("Failure")

Total number of pull requests in the Ramda organization: 1797


In [23]:
# Act upon the returned list of pull requests here without having to call GitHub again

len(resultList)

1797